In [26]:
import spacy
import os
import pandas as pd
from spacy.matcher import Matcher

import re                                                        


import numpy as np
from pandas import ExcelWriter
import matplotlib.pyplot as plt
%matplotlib inline

from gensim.parsing.preprocessing import remove_stopwords
from tqdm import tqdm
import pickle
import string

import os
os.getcwd()

Path = "Privacy_Policies/"
filelist = os.listdir(Path)
txt_files = []
file_names = []
for i in filelist:
    if i.endswith(".txt"):
        file_names.append(i)
        with open(Path + i, 'r') as f:
            txt_files.append(f.read())

## Policy String

In [2]:
Policies = pd.DataFrame([file_names, txt_files]).T
Policies.columns = ["File_name", "text_content"]

In [3]:
Policies

,File_name,text_content
0,InternetSpeedMeter.txt,Internet Speed MeterSearch this siteInternet S...
1,VideoEditor&VideoMaker-InShot.txt,WelcomePrivacy Policy - InShot Inc.Protecting ...
2,Josh-SnackonShortVideoswithTopIndianApp.txt,Privacy PolicyContent PolicyUser AgreementTerm...
3,Telegram.txt,Telegram Privacy Policy EnglishBahasa Indonesi...
4,Dailyhunt-100%IndianAppforNews&Videos.txt,Privacy - DailyhuntAbout UsCareersContact UsAb...
...,...,...
71,"AirtelThanks-Recharge,BillPay,Bank,LiveTV.txt",We at Bharti Airtel Limited and Bharti Hexacom...
72,FLStudioMobile.txt,MOBILE APP PRIVACY | FL Studio FL StudioOvervi...
73,QuickHealTotalSecurity.txt,PRIVACY POLICY- QUICK HEAL MOBILE DEVICE SECUR...
74,DroidCamX.txt,Dev47AppsDroidCam Privacy Policy and Terms of ...


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = cv.fit_transform(Policies['text_content'])
dtm

<76x3520 sparse matrix of type '<class 'numpy.int64'>'
	with 31760 stored elements in Compressed Sparse Row format>

In [5]:
# LDA --------->
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=4,random_state=42)
LDA.fit(dtm)
len(cv.get_feature_names())
len(LDA.components_)

4

In [6]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['application', 'user', 'service', 'including', 'account', 'business', 'time', 'products', 'company', 'gold', 'platform', 'terms', 'customer', 'shall', 'services']


THE TOP 15 WORDS FOR TOPIC #1
['cookies', 'party', 'security', 'using', 'advertising', 'account', 'provide', 'including', 'device', 'access', 'content', 'personal', 'amazon', 'services', 'platform']


THE TOP 15 WORDS FOR TOPIC #2
['settings', 'google', 'personal', 'example', 'device', 'access', 'account', 'including', 'content', 'share', 'facebook', 'products', 'provide', 'data', 'services']


THE TOP 15 WORDS FOR TOPIC #3
['provided', 'contact', 'provide', 'time', 'cookies', 'device', 'used', 'users', 'app', 'party', 'user', 'services', 'service', 'data', 'personal']




In [16]:
dtm.shape

(76, 3520)

In [17]:
topic_results = LDA.transform(dtm)
topic_results.shape

(76, 4)

In [18]:
topic_results[0]

array([0.00313364, 0.00317141, 0.0031853 , 0.99050966])

In [19]:
topic_results[0].round(2)

array([0.  , 0.  , 0.  , 0.99])

In [20]:
Policies['Topic'] = topic_results.argmax(axis=1)

In [21]:
Policies.head(10)

,File_name,text_content,Topic
0,InternetSpeedMeter.txt,Internet Speed MeterSearch this siteInternet S...,3
1,VideoEditor&VideoMaker-InShot.txt,WelcomePrivacy Policy - InShot Inc.Protecting ...,3
2,Josh-SnackonShortVideoswithTopIndianApp.txt,Privacy PolicyContent PolicyUser AgreementTerm...,1
3,Telegram.txt,Telegram Privacy Policy EnglishBahasa Indonesi...,2
4,Dailyhunt-100%IndianAppforNews&Videos.txt,Privacy - DailyhuntAbout UsCareersContact UsAb...,1
5,ShareChat-MadeinIndia.txt,ShareChat Terms of UseLast updated: 22nd Decem...,0
6,PGT+🔧:ProGFX&Optimizer(withadvancesetting).txt,Privacy PolicyYour privacy is important to us....,3
7,MyJio:ForEverythingJio.txt,Reliance Jio - Privacy PolicyMOBILITYJIOFIBERB...,0
8,"AmazonShopping,UPI,MoneyTransfer,BillPayment.txt",Amazon.in Help: Amazon.in Privacy NoticeSkip t...,1
9,GamersGLToolProwithGameTurbo&PingBooster.txt,Privacy PolicyYour privacy is important to us....,3


In [22]:
policies_data = pd.read_csv("Policies.csv")
policies_data.head()

,Unnamed: 0,File_name,text_content,vague_score
0,0,GamersGLToolProwithGameTurbo&PingBooster.txt,Privacy PolicyYour privacy is important to us....,2.566222
1,1,Tasker.txt,Tasker for AndroidTaskerHomeDownloadTaskerNetU...,2.714667
2,2,PandaGamepadPro(BETA).txt,Privacy Policy of Panda Gaming StudioSearch th...,2.855882
3,3,GaanaMusicHindiSongFreeTamilTeluguMP3App.txt,Privacy PolicyThis Privacy Policy explains ou...,2.673979
4,4,WynkMusic-NewMP3HindiTamilSong&PodcastApp.txt,Privacy PolicyData CollectionAt the time you (...,2.754815


In [23]:
policies_data['topic'] =  Policies['Topic']

In [24]:
policies_data.head()

,Unnamed: 0,File_name,text_content,vague_score,topic
0,0,GamersGLToolProwithGameTurbo&PingBooster.txt,Privacy PolicyYour privacy is important to us....,2.566222,3
1,1,Tasker.txt,Tasker for AndroidTaskerHomeDownloadTaskerNetU...,2.714667,3
2,2,PandaGamepadPro(BETA).txt,Privacy Policy of Panda Gaming StudioSearch th...,2.855882,1
3,3,GaanaMusicHindiSongFreeTamilTeluguMP3App.txt,Privacy PolicyThis Privacy Policy explains ou...,2.673979,2
4,4,WynkMusic-NewMP3HindiTamilSong&PodcastApp.txt,Privacy PolicyData CollectionAt the time you (...,2.754815,1


In [25]:
Policies[Policies['Topic'] == 3]

,File_name,text_content,Topic
0,InternetSpeedMeter.txt,Internet Speed MeterSearch this siteInternet S...,3
1,VideoEditor&VideoMaker-InShot.txt,WelcomePrivacy Policy - InShot Inc.Protecting ...,3
6,PGT+🔧:ProGFX&Optimizer(withadvancesetting).txt,Privacy PolicyYour privacy is important to us....,3
9,GamersGLToolProwithGameTurbo&PingBooster.txt,Privacy PolicyYour privacy is important to us....,3
11,SAXVideoPlayer-AllinoneHdFormatpro2021.txt,HomeSearch this sitePrivacy PolicyProtecting y...,3
14,PowerampFullVersionUnlocker.txt,Privacy Policy of Poweramp Music PlayerPrivacy...,3
15,VPNPro-FreeforLifetime.txt,VPNSearch this siteVPNPrivacy Policy For VPN P...,3
16,CrayonIconPack.txt,LineX Privacy PolicySkip to main contentJND AP...,3
17,DiskDiggerProfilerecovery.txt,Privacy Policy for DiskDigger (Android) | Disk...,3
18,PandaGamepadPro(BETA).txt,Privacy Policy of Panda Gaming StudioSearch th...,3


In [27]:
# Anirudh's Network Plot script

instructions = Policies.text_content.values.tolist()

#check = 0
Bigger_list=[]
for sent in instructions:
    #check = check + 1
    #if check <2:
    li = list(sent.split())
    Bigger_list.append(li)
    
with open('train_data.pkl', 'wb') as f:
    pickle.dump(Bigger_list, f)

In [28]:
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm

In [29]:
with open('train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

In [30]:
model = Word2Vec()

In [31]:
model.build_vocab(train_data)

In [32]:
%%time
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 1.3 s, sys: 38 µs, total: 1.3 s
Wall time: 486 ms


(559057, 961950)

In [33]:
model.save('w2v.model')

In [36]:
search_term = 'share'
sim_terms = model.wv.most_similar(search_term, topn=5)
sim_terms

[('information', 0.9871485829353333),
 ('personal', 0.9848772287368774),
 ('disclose', 0.9817430973052979),
 ('provide', 0.9750065803527832),
 ('use', 0.97100830078125)]